### **■ 유통 판매량 예측 및 재고 최적화**
# **단계 2: 모델링을 위한 데이터 전처리**

<img src = "https://github.com/Jangrae/img/blob/master/store.png?raw=true" width=800, align="left"/>

# **⏰ 수행 과제**

다음과 같은 과정으로 프로젝트를 진행합니다.

#### **1. 환경 설정**
- 이후 진행에 필요한 환경 설정을 수행합니다.

#### **2. 데이터 셋 구성하기 #1**
- 44번 매장의 데이터만을 대상으로 합니다.
- 대표 상품(3, 12, 42) 데이터만을 대상으로 합니다.
- 상품별로 데이터셋을 각각 구성해야 합니다.

#### **3. 1차 모델링**
- Linear Regression 알고리즘을 사용해 모델링합니다.
- 모델의 검증 성능을 평가하고 기록합니다.

#### **4. 데이터 셋 구성하기 #2**
- 모델의 성능을 높이기 위해 의미있는 변수를 추가합니다.
- 상품코드 별로 각각 데이터셋을 구성해야 합니다.
- 상품별 데이터 셋 이름은 다음과 같이 통일합니다.

#### **5. 2차 모델링**
- Linear Regression 알고리즘으로 다시 모델링합니다.
- 모델 성능을 높이기 위해 적절한 변수를 추가합니다.
- 데이터 셋에 변화를 주면서 변화되는 성능을 기록하고 비교합니다.

#### **6. 3차 모델링**
- Random Forest, LightGBM 알고리즘으로 모델링하고 성능을 기록합니다.
- 하이퍼파라미터 튜닝은 하지 않습니다(모든 파라미터 기본값 사용).
- 우선 Linear Regression 모델보다 성능이 좋은지 비교하는 데 의미를 둡니다.

#### **7. 데이터 저장**
- 이후 실습을 위해 최종 구성된 데이터프레임을 파일로 저장합니다.

# **1. 환경 설정**

- 이후 진행에 필요한 환경 설정을 수행합니다.

## **(1) 경로 설정**

- 프로젝트 수행 환경에 맞게 파일 경로를 설정합니다.

### **1) 로컬 수행(Anaconda)**
- project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [ ]:
# 기본 경로
path = ''

### **2) 구글 콜랩 수행**

- 구글 콜랩을 사용중이면 구글 드라이브를 연결합니다.

In [ ]:
# 구글 드라이브 연결, 패스 지정
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/project/'

## **(2) 라이브러리 불러오기**

- 이후 사용할 라이브러리를 모두 불러옵니다.

In [ ]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

- 모델이 예측한 결과를 시각화할 때 다음 함수를 활용합니다.

In [ ]:
# 함수 만들기
def plot_model_result(y_train, y_val, y_pred):
    
    y_train = pd.Series(y_train)
    y_val = pd.Series(y_val)
    y_val.index = range(len(y_train), len(y_train) + len(y_val))

    y_pred = pd.Series(y_pred.reshape(-1,), index=y_val.index)

    # 전체 시각화
    plt.figure(figsize=(12, 5))
    plt.subplot(2,1,1)
    plt.plot(y_train, label='y_train', color='tab:blue')
    plt.plot(y_val, label='y_val', color='tab:green')
    plt.plot(y_pred, label='y_pred', color='tab:orange')
    plt.legend()
    plt.subplot(2,1,2)
    plt.plot(y_val, label = 'y_val', color='tab:green')
    plt.plot(y_pred, label = 'y_pred', color='tab:orange')
    plt.legend()
    plt.tight_layout()
    plt.show()

## **(3) 데이터 불러오기**

- 이후 분석 대상이 되는 파일을 불러오고 기본 정보를 확인합니다.

### **1) 데이터 불러오기**

- 분석 대상 데이터를 불러옵니다.
- 데이터프레임 이름은 다음과 같이 통일합니다.
    - sales: 판매 정보
    - orders: 고객 방문수
    - oil_price: 휘발유 가격
    - stores: 매장 정보
    - products: 상품 정보
- 날짜 데이터를 갖는 Date 변수는 편의를 위해 datetime 형으로 변경합니다.

In [ ]:
# 데이터 불러오기
sales = pd.read_csv(path + 'sales_train.csv')
orders = pd.read_csv(path + 'orders_train.csv')
oil_price = pd.read_csv(path + 'oil_price_train.csv')
stores = pd.read_csv(path + 'stores.csv')
products = pd.read_csv(path + 'products.csv')

In [ ]:
# datetime 형으로 변환
sales['Date'] = pd.to_datetime(sales['Date'] )
oil_price['Date'] = pd.to_datetime(oil_price['Date'] )
orders['Date'] = pd.to_datetime(orders['Date'] )

### **2) 기본 정보 확인**

- 각 데이터의 기본 정보를 확인합니다.

# **2. 데이터 셋 구성하기 #1**

- 44번 매장의 데이터만을 대상으로 합니다.
- 대표 상품(3, 12, 42) 데이터만을 대상으로 합니다.
- 상품별로 데이터셋을 각각 구성해야 합니다.
- 상품별 데이터프레임 이름은 다음과 같이 통일합니다.
    - data03: 3번 상품 데이터 셋
    - data12: 12번 상품 데이터 셋
    - data42: 42번 상품 데이터 셋
- 우선 다음과 같은 변수만을 갖는 기본 데이터 셋을 구성합니다.
    - Date: 날짜
    - Qty: 판매량
    - CustomerCount: 고객 방문수
    - WeekDay: 요일(Monday ~ Sunday)
    - Month: 월(1 ~ 12)
    - WTI_Price: 최근 14일 동안의 휘발유 가격 평균
    - Target: 예측해야 하는 판매량
- 참고: 요일과 월은 다음 형태로 얻을 수 있습니다.

~~~
df['요일'] = df['날짜'].dt.day_name()
df['월'] = df['날짜'].dt.month
~~~

- 참고: 최근 14일 간의 가격 평균은 다음 형태로 얻을 수 있습니다.

~~~
df['가격'].rolling(14, min_periods=1).mean()
~~~
- Target 즉, 예측해야 하는 판매량은 상품의 LeadTime 이후의 판매량입니다.
- 예를 들어 5월 2일, 상품 LeadTIme이 2이면 5월 4일 판매량이 예측해야 하는 판매량입니다.

## **(1) 상품: 3 - Beverage**

- 3번 상품 데이터 셋을 구성합니다.
- 데이터프레임 이름은 **data03**으로 합니다.

## **(2) 상품: 12 - Milk**

- 12번 상품 데이터 셋을 구성합니다.
- 데이터프레임 이름은 **data012**으로 합니다.

## **(3) 상품: 42 - Agricultural products**

- 42번 상품 데이터 셋을 구성합니다.
- 데이터프레임 이름은 **data42**으로 합니다.

# **3. 1차 모델링: 기준 모델**

- Linear Regression 알고리즘을 사용해 모델링합니다.
- 모델링 과정에서 변수 이름은 **x_train, x_val, y_train, y_val, y_pred**을 사용합니다.
- **검증용 데이터**는 학습용 데이터에서 **최근 120일간**의 데이터를 사용합니다.
- 모델의 검증 성능을 평가하고 기록합니다.

## **(1) 상품: 3 - Beverage**

- 3번 상품에 대해 모델링하고 성능을 평가합니다.
- 모델 이름은 **model03_lin**로 합니다.

- 모델이 예측한 결과를 시각화해 확인합니다.

## **(2) 상품: 12 - Milk**

- 12번 상품에 대해 모델링하고 성능을 평가합니다..
- 모델 이름은 **model12_lin**로 합니다.

- 모델이 예측한 결과를 시각화해 확인합니다.

## **(3) 상품: 42 - Agricultural products**

- 42번 상품에 대해 모델링하고 성능을 평가합니다..
- 모델 이름은 **model42_lin**로 합니다.

- 모델이 예측한 결과를 시각화해 확인합니다.

# **4. 데이터 셋 구성하기 #2**

- 모델의 성능을 높이기 위해 의미있는 변수를 추가합니다.
- 상품코드 별로 각각 데이터셋을 구성해야 합니다.
- 상품별 데이터 셋 이름은 다음과 같이 통일합니다.
    - data03: 3번 상품 데이터 셋
    - data12: 12번 상품 데이터 셋
    - data42: 42번 상품 데이터 셋
- 예를 들면 다음과 같은 변수들을 갖는 데이터 셋을 구성할 수 있습니다.
    - Date: 날짜
    - Qty: 판매량
    - CustomerCount: 고객 방문수
    - WeekDay: 요일
    - Month: 월
    - WTI_Price: 유가
    - **Category_Qty: 동일 카테고리 판매량 합계**
    - **City_CustCount: 동일 지역 방문객 수**
    - **Qty_Lag_1: 1일 전 판매량**
    - **Qty_Lag_7_mean: 최근 7일간 판매량 평균**
    - **기타 등등...**
    - Target: 예측해야 하는 판매량

## **(1) 상품: 3 - Beverage**

- 3번 상품 데이터 셋을 구성합니다.
- 데이터프레임 이름은 **data03**으로 합니다.

## **(2) 상품: 12 - Milk**

- 12번 상품 데이터 셋을 구성합니다.
- 데이터프레임 이름은 **data12**으로 합니다.

## **(3) 상품: 42 - Agricultural products**

- 42번 상품 데이터 셋을 구성합니다.
- 데이터프레임 이름은 **data42**으로 합니다.

# **5. 2차 모델링: 모델 성능 변화 확인**

- Linear Regression 알고리즘으로 다시 모델링합니다.
- 모델링 과정에서 변수 이름은 **x_train, x_val, y_train, y_val, y_pred**을 사용합니다.
- 모델 성능을 높이기 위해 적절한 변수를 추가합니다.
- 데이터 셋에 변화를 주면서 변화되는 성능을 기록하고 비교합니다.

## **(1) 상품: 3 - Beverage**

- 3번 상품에 대해 모델링하고 성능을 평가합니다..
- 모델 이름은 **model03_lin**로 합니다.

- 모델이 예측한 결과를 시각화해 확인합니다.

## **(2) 상품: 12 - Milk**

- 12번 상품에 대해 모델링하고 성능을 평가합니다..
- 모델 이름은 **model03_lin**로 합니다.

- 모델이 예측한 결과를 시각화해 확인합니다.

## **(3) 상품: 42 - Agricultural products**

- 42번 상품에 대해 모델링하고 성능을 평가합니다..
- 모델 이름은 **model42_lin**로 합니다.

- 모델이 예측한 결과를 시각화해 확인합니다.

# **6. 3차 모델링: 초기 모델 생성**

- Random Forest, LightGBM 알고리즘으로 모델링하고 성능을 기록합니다.
- 모델링 과정에서 변수 이름은 **x_train, x_val, y_train, y_val, y_pred**을 사용합니다.
- 하이퍼파라미터 튜닝은 하지 않습니다(모든 파라미터 기본값 사용).
- **검증용 데이터**는 학습용 데이터에서 **최근 120일간**의 데이터를 사용합니다.
- 우선 Linear Regression 모델보다 성능이 좋은지 비교하는 데 의미를 둡니다.

## **(1) Random Forest 모델**

### **1) 상품: 3 - Beverage**

- 3번 상품에 대해 모델링하고 성능을 평가합니다..
- 모델 이름은 **model03_rdf**로 합니다.

- 모델이 예측한 결과를 시각화해 확인합니다.

### **2) 상품: 12 - Milk**

- 12번 상품에 대해 모델링하고 성능을 평가합니다..
- 모델 이름은 **model12_rdf**로 합니다.

- 모델이 예측한 결과를 시각화해 확인합니다.

### **3) 상품: 42 - Agricultural products**

- 42번 상품에 대해 모델링하고 성능을 평가합니다..
- 모델 이름은 **model42_rdf**로 합니다.

- 모델이 예측한 결과를 시각화해 확인합니다.

## **(2) LightGBM 모델**

### **1) 상품: 3 - Beverage**

- 3번 상품에 대해 모델링하고 성능을 평가합니다..
- 모델 이름은 **model03_lgb**로 합니다.

- 모델이 예측한 결과를 시각화해 확인합니다.

### **2) 상품: 12 - Milk**

- 12번 상품에 대해 모델링하고 성능을 평가합니다..
- 모델 이름은 **model12_lgb**로 합니다.

- 모델이 예측한 결과를 시각화해 확인합니다.

### **3) 상품: 42 - Agricultural products**

- 42번 상품에 대해 모델링하고 성능을 평가합니다..
- 모델 이름은 **model42_lgb**로 합니다.

- 모델이 예측한 결과를 시각화해 확인합니다.

# **7. 데이터프레임 저장**

- 이후 실습을 위해 최종 구성된 데이터프레임을 파일로 저장합니다.
- joblib룰 사용해 저장할 때는 일반적으로 pkl 확장자를 붙입니다.
- 다음과 같은 형태의 구문으로 저장합니다.
~~~
joblib.dump(mydata, path + 'mydata.pkl')
~~~
- 파일 이름은 다음과 같이 통일합니다.
    - data03.pkl: 3번 상품 데이터 셋
    - data12.pkl: 12번 상품 데이터 셋
    - data42.pkl: 42번 상품 데이터 셋

In [ ]:
# 파일 저장
joblib.dump(data03, path + 'data03.pkl')
joblib.dump(data12, path + 'data12.pkl')
joblib.dump(data42, path + 'data42.pkl')